# Analysis movies

In [1]:
#import 
%reload_ext autoreload
%autoreload 2
import pandas as pd 
import numpy as np
import get_files as gf

# Bonus 2: Récupérer et extraire le jeu de données fourni de manière automatique

In [2]:
url_file ='https://stdatalake005.blob.core.windows.net/public/movies_dataset.zip'
gf.download_file(url_file)

data.zip existe déja
data existe déja


# Chargement des données 


In [3]:

# chagement du fichier name.basics.tsv
df_name_basics= pd.read_csv('../../data/name.basics.tsv', sep ='\t', na_values = ["\\N", "nan"])


# chargement du fichier title.principals.tsv
df_title_principals = pd.read_csv('../../data/title.principals.tsv',sep ='\t', na_values = ["\\N", "nan"])


# chargement du fichier title.basics.tsv
df_title_basics = pd.read_csv('../../data/title.basics.tsv', sep ='\t', na_values = ['\\N', 'nan'])

# chargement du fichier title.akas.tsv
df_title_akas = pd.read_csv('../../data/title.akas.tsv', delimiter ='\t', na_values =["\\N", "nan"])


# chargement du fichier title.ratings.tsv
df_title_ratings= pd.read_csv('../../data/title.ratings.tsv', sep ='\t', na_values = ["\\N", "nan"])




# 1. Liste des Acteurs par Films

In [4]:

df_name_basics = df_name_basics.drop_duplicates()
df_name_basics_filter = df_name_basics.filter(['nconst','primaryName'])

df_title_principals = df_title_principals.drop_duplicates()
df_title_principals_filter = df_title_principals.filter(['tconst','nconst', 'category'])

df_title_principals_actor = df_title_principals_filter.loc[(df_title_principals_filter['category']== 'actor') | (df_title_principals_filter['category']== 'actress')]
df_title_principals_actor = df_title_principals_actor.filter(['tconst','nconst'])

df_title_basics = df_title_basics.drop_duplicates()
df_title_basics_movies = df_title_basics[(df_title_basics.titleType == "movie") | (df_title_basics.titleType == "tvMovie")]
df_title_basics_movies = df_title_basics_movies.filter(['tconst','originalTitle'])

df_merge_actor = pd.merge(df_name_basics_filter,df_title_principals_actor, on = "nconst", how ='inner')
df_merge_actor_movie = pd.merge(df_merge_actor,df_title_basics_movies, on="tconst", how ="inner")
df_merge_actor_movie_filter = df_merge_actor_movie[["originalTitle","primaryName"]]
df_merge_actor_movies = df_merge_actor_movie_filter.groupby(['originalTitle'])['primaryName'].apply(','.join).reset_index()

df_merge_actor_movies.to_csv('../OUT/actor_movies.csv', sep ='\t')

del(df_name_basics_filter,df_title_principals_filter,df_title_principals_actor,df_title_basics_movies,df_merge_actor,df_merge_actor_movie_filter,df_merge_actor_movies)


# 2. Liste des Films Américains 

In [6]:
# get only movies distriuted in US
df_akas_us = df_title_akas.loc[df_title_akas['region'] == "US"].filter(['titleId', 'title', 'region'])
df_akas_us.rename(columns = {'title': 'title_us', 'region': 'region_us'}, inplace = True)

# get only movies distriuted in FR
df_akas_fr = df_title_akas.loc[df_title_akas['region'] == "FR"].filter(['titleId', 'title', 'region'])
df_akas_fr.rename(columns = {'title': 'title_fr', 'region': 'region_fr'}, inplace = True)

# merge movies distributed in US and in FR
df_akas_merged = pd.merge(df_akas_us, df_akas_fr, how='left', on='titleId')

# create new column "is_title_fr" to describe if movie has english title or only french title
df_akas_merged['is_title_fr'] = np.where(df_akas_merged['title_fr'].isnull(), False, True)

# remove US title if FR title exists
df_akas_merged['title_us'] = np.where(df_akas_merged['title_fr'].notnull(), '', df_akas_merged['title_us'])

# merge with rating
df_rating_akas = pd.merge(df_akas_merged, df_title_ratings, how='inner', right_on='tconst', left_on='titleId')

# remove columns
df_rating_akas = df_rating_akas.filter(['titleId', 'title_fr', 'title_us', 'is_title_fr', 'averageRating'])
# generate csv file
df_rating_akas.to_csv('../OUT/american_movies.csv',sep ='\t')

del(df_akas_fr, df_akas_us, df_akas_merged, df_rating_akas)


# 3. Les notes moyennes des différents genres

In [8]:
# split column gere by ','
df_title_basic_filter = df_title_basics[['tconst', 'genres']].dropna()
df_title_basic_genre = pd.DataFrame(df_title_basic_filter.genres.str.split(',').tolist(), index = df_title_basic_filter.tconst).stack()

# define 'tconst' as index
df_title_basic_genre = df_title_basic_genre.reset_index([0, 'tconst'])

# rename columns
df_title_basic_genre.columns = ['tconst', 'genre']

# merge new df with rating dataframe
merge_genre_rating = pd.merge(df_title_basic_genre, df_title_ratings, how = 'inner', on = 'tconst')

# keep only titles 'genre' and 'averageRating'
merge_genre_rating = merge_genre_rating.filter(['genre', 'averageRating'])

# calcul of average by genre
avg_rating_genre = merge_genre_rating.groupby('genre').mean('averageRating')

avg_rating_genre = avg_rating_genre['averageRating'].round(decimals=2)

# avg_rating_genre to csv
avg_rating_genre.to_csv('../OUT/genre_average.csv',sep ='\t')

del(df_title_basic_filter,df_title_basic_genre,merge_genre_rating,avg_rating_genre)

# 4. Note moyenne des acteurs par rapport aux films

In [5]:

df_merge_actor_moy= df_merge_actor_movie[["primaryName","originalTitle","tconst"]]

df_title_ratings_filter = df_title_ratings.filter(['tconst', 'averageRating'])

df_merge_actor_moy_note = pd.merge(df_merge_actor_moy,df_title_ratings_filter, on ='tconst', how = 'inner')
df_merge_actor_moy_note = df_merge_actor_moy_note.groupby(['primaryName'])['averageRating'].mean()

df_merge_actor_moy_note.to_csv('../OUT/actor_average.csv',sep ='\t')







# Bonus 1: Générer des recommandations de nouveaux films en fonction de l'historique de l'utilisateur.

#### Creer un fichier tsv qui contient des utilisateurs et leurs notes sur des films, les films et les notes sont recupérés aléatoirement
la procédure est la suivante:  
1. creer une liste d'utilisateurs avec un historique de films notés  
2. separer les genres de chaque film si un film est de plusieurs genres  
3. merger les films avec leurs genres(on va avoir des lignes dupliquées de films mais avec genre differents)  
4. associer apres, chaque film avec son rating  
5. récuperer les utilisateurs avec le genre le mieux notés  
6. applicquer la formule Weighted Average Ratings de IMDB, sa formule est:  
WeightedRating(WR)=((v/v+m) \* R) + ((m/v+m) \* C)  
7. trier les films par les mieux notés  
8. enregistrer dans un fichier CSV

In [ ]:
# recuperer les id des titres dans une liste
movie_list = df_title_basics['tconst'].to_list()

# creer une dataframe pour 
df = pd.DataFrame(columns=['userId', 'tconst', 'averageRating'])
index = 0

# remplir le dataframe par l'historique des utilisateurs
for i in range(1, 11):
    for j in range(1, 21):
        index += 1
        df.loc[index, 'userId'] = i
        df.loc[index, 'tconst'] = random.choice(movie_list)
        df.loc[index, 'averageRating'] = format(random.uniform(3, 9), '.1f')

# enregistrer le fichier
df.to_csv('../../data/users.tsv', sep ='\t', index = False)

# creer le dataset user
df_users = pd.read_csv('../../data/users.tsv', sep ='\t')

# definir un dataset de movies pour le bonus 1
df_title_basics_bonus_1 = df_title_basics[['tconst', 'primaryTitle', 'genres']]

# supprimer les valeurs nulle
df_title_basics_na = df_title_basics_bonus_1.dropna()
df_title_basics_cleaned = df_title_basics_na[df_title_basics_na['genres'] != '\\N']

# dupliquer les lignes par genres
df_title_by_genre = pd.DataFrame(
    df_title_basics_cleaned.genres.str.split(',').tolist(), 
    index = df_title_basics_cleaned.tconst).stack()

# redefinir l'index
df_title_by_genre = df_title_by_genre.reset_index([0, 'tconst'])

# rennomer les comonnes
df_title_by_genre.columns = ['tconst', 'genre']

# merger les films avec leurs genres
df_title_splited = pd.merge(
    df_title_basics_bonus_1.filter(['tconst' , 'primaryTitle']),
    df_title_by_genre, 
    how = 'inner', 
    on = 'tconst')

# merger les films avec leurs ratings
df_title_rating_merged = pd.merge(
    df_title_splited,
    df_title_ratings,
    how = 'inner', 
    on = 'tconst'
)

# recuperer le genre le mieux notés par chaque utilisateur
df_users_merged = pd.merge(df_users, df_title_by_genre, how='inner', on='tconst')
df_users_best_notes = df_users_merged.groupby(['userId','genre'])['averageRating'].mean()
df_users_best_notes = df_users_best_notes.reset_index()
df_users_best_notes = df_users_best_notes.sort_values(by=['userId', 'averageRating'], ascending=[1, 0])
df_users_best_notes = df_users_best_notes.groupby('userId').head(1)

# Fonction qui permet de calculer le weighted rating pour chaque film
# v: nombre de votes pour le film
# R: note moyenne pour un film
# m: nombre de vote minimal pour que le film soit evalué
# C: la valeur moyenne des votes
def weighted_rating(x, m=m, C=C):
    v = x['numVotes']
    R = x['averageRating']
    
    return (v/(v+m) * R) + (m/(m+v) * C)

# calcul de la valeur moyenne des votes
C = df_title_rating_merged['averageRating'].mean()

# Calcul du nombre minimum de votes requis pour figurer dans le tableau. 
# on a definit 90% comme seuil
# pour qu'un film figure dans les classements, il doit avoir plus de votes qu'au moins 90% des films de la liste.
m = df_title_rating_merged['numVotes'].quantile(0.90)

# garder seulement les films que leurs nombre de votes et supperieur à la seuil
q_movies = df_title_rating_merged.copy().loc[df_title_rating_merged['numVotes'] >= m]

# ajouter une colonne Score qui contiendra la valeur retournée par la fonction `weighted_rating()`
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

# trier les films par score
q_movies = q_movies.sort_values('score', ascending=False)

#Print the top 15 movies
q_movies[['tconst', 'primaryTitle', 'numVotes', 'averageRating', 'genre', 'score']].head(20)

# recuperer le film le mieux évalué dans chaque genre
best_movies_score = q_movies.groupby(['genre', 'tconst'], sort=False)['score'].max().reset_index()
best_movies_score = best_movies_score.groupby('genre').head(1)

# creer dataframe films recommendés
# recommendé un film à chaque utilisateur en se basanr le genre le mieux noté par chaque utilisateur
df_users_recommended_movies = pd.merge(df_users_best_notes, best_movies_score, how='inner', on='genre')
recommended_movies = pd.merge(df_users_recommended_movies, df_title_basics_bonus_1, how='inner', on='tconst')

# ecriture du fichier
recommended_movies['score'] = (recommended_movies['score'].round(decimals=2) * 10)
recommended_movies[['userId', 'primaryTitle', 'genre', 'score']].to_csv('../OUT/recommended_movies.tsv', sep ='\t', index = False)